<a href="https://colab.research.google.com/github/TianrunCheng/hello-world/blob/master/dp04train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install transformers

In [ ]:
!pip install pytorch_pretrained_bert

In [ ]:
import os, sys, time, datetime, re
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from collections import Counter
import torch
from torch.utils.data import TensorDataset, DataLoader


use_cuda = torch.cuda.is_available()
def cuda(arr):
    if use_cuda:
        return arr.cuda()
    return arr


from pytorch_pretrained_bert.tokenization import BertTokenizer, WordpieceTokenizer
from pytorch_pretrained_bert.modeling import BertForPreTraining, BertPreTrainedModel, BertModel, BertConfig, BertForMaskedLM, BertForSequenceClassification
# from pathlib import Path
# import torch
# import re
from torch import Tensor
from torch.nn import BCEWithLogitsLoss
# from fastai.text import Tokenizer, Vocab

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from pytorch_pretrained_bert.optimization import BertAdam

train = pd.read_csv('train.csv')
print('Total train num: ', len(train))
train = train.dropna()
print('Valid train num: ', len(train))

train_textID = train.textID.tolist()
train_text = train.text.tolist()
train_selected_text = train.selected_text.tolist()
train_sentiment = train.sentiment.tolist()

print(train_text[:3])
print(train_selected_text[:3])
print(train_sentiment[:10])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Total train num:  27481
Valid train num:  27480
[' I`d have responded, if I were going', ' Sooo SAD I will miss you here in San Diego!!!', 'my boss is bullying me...']
['I`d have responded, if I were going', 'Sooo SAD', 'bullying me']
['neutral', 'negative', 'negative', 'negative', 'negative', 'neutral', 'positive', 'neutral', 'neutral', 'positive']


In [ ]:
"""
Use Bert-tokenizer to tokenize train_text and train_selected_text
Use Bert's dictionary to convert tokens to indexes
Prepare training data to be ready to feed into BERT:

    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])  #required
    ### [batch_size, sequence_len], idx of input sequences
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    ### [batch_size, sequence_len] used when input sequence is smaller than max seq len in this batch
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
    ### [batch_size, sequence_len], '0': sentence_A, '1': sentence_B

"""

# NUM of datapoints to use, max is 27480
data_num = 27480
validation_rate = 0.2 
fixed_input_size = 200
#############################

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


def text_to_ids(sentense):
  tokens = tokenizer.tokenize(sentense)
  tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
  return tokens_ids

#
# padding each string to a default length, as in var: fixed_input_size
# Also convert the format from list to np.darray
def padding(s):
  return np.array( s+ [0]*(fixed_input_size-len(s)) )

input_idx = []
input_mask = []
output_idx = []
for j in range( data_num ):
  # if j<10:
    # print(tokenizer.tokenize("[SEP]"))
  CLS_id = text_to_ids("[CLS]")
  sentiment_id = text_to_ids(train_sentiment[j])
  SEP_id = text_to_ids("[SEP]")
  train_text_ids = text_to_ids(train_text[j])
  # token([CLS] + sentiment_label + [SEP] + Train_text)
  tokens_ids = CLS_id + sentiment_id + SEP_id + train_text_ids 
  input_idx.append(padding(tokens_ids))

  # input_idx = [x, x, ..., x] 512
  # real_input_idx = [x, ..., x] 200
  #                  [sequence+pad]
  # real_token_type = [0, 0, 0, 1,1,...,1] 200
  # real_input_mask = [1,...,1,0,...,0] 200
  #                   [sequence+pad]

  mask = [1]* (len(train_text[j])+3)
  input_mask.append(padding(mask))

  
  subset_tokens_ids = text_to_ids(train_selected_text[j])
  whole_token_ids = text_to_ids(train_text[j])
  l = len(subset_tokens_ids)
  L = len(whole_token_ids)
  output= [0]*2
  j = 0
  for j in range(L):
    if whole_token_ids[j:j+l] == subset_tokens_ids:
      # put 1 on all positions for the first occurence of the given substring
      output+= [1]*l
      break
    else:
      output+=[0] 
  output_idx.append(padding(output))







In [ ]:

######################################################## crucial constants
#input length to be fed into BERT
# fixed_input_size = 200

# max 27840
train_size = 24000 
valid_size = 3000

batch_size = 32

dropout_prob = 0.4
hidden_size = 768
########################

nEpochs = 12
learning_rate = 0.00002
L2_lambda = 0.0

############################################################


train_data = TensorDataset(torch.from_numpy(np.array(input_idx[:train_size])),\
              torch.from_numpy(np.array(input_mask[:train_size])),\
               torch.from_numpy(np.array(output_idx[:train_size])))
train_loader = DataLoader(train_data, shuffle=True, batch_size= batch_size)

validation_data = TensorDataset(torch.from_numpy(np.array(input_idx[-valid_size:])),\
              torch.from_numpy(np.array(input_mask[-valid_size:])),\
               torch.from_numpy(np.array(output_idx[-valid_size:])),\
              torch.from_numpy(np.array( range(len(input_idx)-valid_size,len(input_idx)) ))\
              )
validation_loader = DataLoader(validation_data, shuffle=True, batch_size= batch_size)



class Model(BertPreTrainedModel):
  def __init__(self,config):

    super(Model,self).__init__(config)
    self.bert = BertModel(config)
    self.dropout = torch.nn.Dropout(dropout_prob)
    self.ll = torch.nn.Linear(hidden_size, 2)
    self.apply(self.init_bert_weights)
    for param in self.bert.parameters():
      param.requires_grad = True 

  def forward(self, input_ids, attention_mask=None, labels=None ):
    # token_type_ids = torch.zeros([len(input_ids), fixed_input_size], dtype=torch.long) # wrong input!
    type_line = [1,1,1]
    for i in range(fixed_input_size-3):
      type_line.append(0)
    token_type_ids = []
    for j in range(len(input_ids)):
      token_type_ids.append(type_line)
    token_type_ids = torch.tensor(token_type_ids, dtype=torch.long)
    token_type_ids = cuda(token_type_ids)
    output,pooled_output = self.bert(input_ids, token_type_ids=token_type_ids,\
                                     attention_mask=attention_mask, output_all_encoded_layers=False)
    output = self.dropout(output)
    output = self.ll(output)
    return output
  
model = Model.from_pretrained("bert-base-uncased")
print(model)

Model(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
 

In [ ]:

def train_rnn_model_1(model, data_loader):

    model = model.cuda()
    criterion = torch.nn.CrossEntropyLoss(reduction='sum')
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=L2_lambda)

    train_loss = 0
    trained_sen = 0

    model.train()
    for epoch in range(nEpochs):
        for idx, (input_idx, attention_mask, target) in enumerate(data_loader):
            input_idx=torch.tensor(input_idx,dtype=torch.long)
            attention_mask = torch.tensor(attention_mask,dtype=torch.long)
            target=torch.tensor(target,dtype=torch.long)
            input_idx, attention_mask, target = cuda(input_idx),cuda(attention_mask),cuda(target)
            model.zero_grad()
            out = model(input_idx, attention_mask)
            
            #loss = criterion(out, target)
            loss = 0
            for i in range(len(input_idx)):
                loss += criterion(out[i][:200,:200], target[i][:200])
            
            loss.backward()
            optimizer.step()

            train_loss += loss.data.item()
            trained_sen += len(input_idx)
            
            if idx % 5 == 0:
                print("%2d %6d %8.3f" % (epoch, idx*batch_size, train_loss/trained_sen))
            
            # torch.cuda.empty_cache()

        # model_name = "model"+str(epoch)+'_'+ str(train_loss/trained_sen)+'.pth'
        model_save_name = "model"+str(epoch)+'_'+ str(train_loss/trained_sen)+'.pth'
        path = F"/content/drive/My Drive/{model_save_name}"  
        torch.save(model.state_dict(), path)
        # torch.save(model, model_name)
                
    torch.save(model, "model_final.pth")

# torch.cuda.empty_cache()
train_rnn_model_1(model, train_loader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


 0      0  153.688
 0    160   52.290
 0    320   36.385
 0    480   30.335
 0    640   26.630
 0    800   24.146
 0    960   22.516
 0   1120   21.243
 0   1280   20.253
 0   1440   19.458
 0   1600   18.921
 0   1760   18.391
 0   1920   17.944
 0   2080   17.595
 0   2240   17.301
 0   2400   16.990
 0   2560   16.771
 0   2720   16.557
 0   2880   16.388
 0   3040   16.211
 0   3200   16.024
 0   3360   15.904
 0   3520   15.768
 0   3680   15.615
 0   3840   15.454
 0   4000   15.224
 0   4160   14.938
 0   4320   14.711
 0   4480   14.482
 0   4640   14.281
 0   4800   14.070
 0   4960   13.902
 0   5120   13.710
 0   5280   13.504
 0   5440   13.356
 0   5600   13.219
 0   5760   13.069
 0   5920   12.953
 0   6080   12.793
 0   6240   12.697
 0   6400   12.594
 0   6560   12.456
 0   6720   12.331
 0   6880   12.236
 0   7040   12.139
 0   7200   12.038
 0   7360   11.935
 0   7520   11.820
 0   7680   11.722
 0   7840   11.624
 0   8000   11.536
 0   8160   11.485
 0   8320   

KeyboardInterrupt: ignored

In [ ]:
import torch.nn as nn
model.load_state_dict(torch.load("/content/drive/My Drive/model6_5.193322490601313.pth"))
# model = torch.load("/content/drive/My Drive/model6_5.138287643682389.pth")
model.eval()
def jaccard(list1, list2):
    a = set(list1)
    b = set(list2)
    c = a.intersection(b)
    if (len(a) + len(b) - len(c))==0:
        print('--------error:',len(a), len(b), len(c))
        return 0
    return float(len(c)) / (len(a) + len(b) - len(c))

def getWord(val_ls):
    rst = []
    for val in val_ls:
        if val <= 2:
            continue
        ky = list(vocab_to_int.keys())[list(vocab_to_int.values()).index(val)]
        rst.append(ky)
    return rst

def getTarget(val_ls, label_ls):
    rst = []
    for i in range(len(val_ls)):
        if label_ls[i] == 1:
            rst.append(val_ls[i])
    return rst

def out_to_class(out):
    m = nn.Softmax(2)
    a = m(out).cpu()
    a= a.detach().numpy()
    return np.argmax(a,axis=-1)

def ids_to_string(ids):
  output_substring = " ".join( tokenizer.convert_ids_to_tokens(ids))
  fine_output_substring = output_substring.replace(' ##', '')
  return fine_output_substring
def boolean_to_string(answer_boolean,orig_ids):
   ans_ids  = []
   for j,out in enumerate(answer_boolean):
    if out ==1:
      ans_ids.append(orig_ids[j+1])
   return ids_to_string(ans_ids)


with torch.no_grad():
  score = 0.0
  check = True
  for idx, (orin_input_idx, attention_mask, output_boolean, indices) in enumerate(validation_loader):

      input_idx=torch.tensor(orin_input_idx,dtype=torch.long)
      attention_mask = torch.tensor(attention_mask,dtype=torch.long)
      input_idx, attention_mask = cuda(input_idx),cuda(attention_mask)
      out = model(input_idx, attention_mask)
      answers = out_to_class(out)
      
      for i,answer in enumerate(answers):
        orig_ids = orin_input_idx[i].numpy()
        output = boolean_to_string(answer, orig_ids)
        score += jaccard(output, train_selected_text[indices[i]])

        if check:
          print(ids_to_string(orin_input_idx[i].numpy()))
          print("######Our output##########")
          print(boolean_to_string(answer, orig_ids))
          print("#####Target######")
          print(train_selected_text[indices[i]])
          #print(boolean_to_string(output_boolean[i],orig_ids))
          print("#######END#######")
      check = False 
      #break 
  print(score)
  print(score/float(valid_size))

  # print(err_count)
  # print(total_count, 1 - err_count/total_count)
  # # print(score/len(test_x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[CLS] positive [SEP] courtney seems like a cool chick & shes pretty [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
test = pd.read_csv('test.csv')

test = test.dropna()


test_textID = test.textID.tolist()
test_text = test.text.tolist()
test_sentiment = test.sentiment.tolist()

input_idx = []
input_mask = []
output_idx = []
for j in range( len(test_text) ):

  CLS_id = text_to_ids("[CLS]")
  sentiment_id = text_to_ids(test_sentiment[j])
  SEP_id = text_to_ids("[SEP]")
  test_text_ids = text_to_ids(test_text[j])
  
  tokens_ids = CLS_id + sentiment_id + SEP_id + test_text_ids 
  input_idx.append(padding(tokens_ids))

  
  mask = [1]* (len(test_text[j])+3)
  input_mask.append(padding(mask))


test_data = TensorDataset(torch.from_numpy(np.array(input_idx)),\
              torch.from_numpy(np.array(input_mask)),\
               )
test_loader = DataLoader(test_data, shuffle=False, batch_size= batch_size)

with torch.no_grad():

  outputs= []
  score = 0.0
  check = False 
  for idx, (orin_input_idx, attention_mask) in enumerate(test_loader):
      input_idx=torch.tensor(orin_input_idx,dtype=torch.long)
      attention_mask = torch.tensor(attention_mask,dtype=torch.long)
      input_idx, attention_mask = cuda(input_idx),cuda(attention_mask)
      out = model(input_idx, attention_mask)
      answers = out_to_class(out)
      
      for i,answer in enumerate(answers):
        orig_ids = orin_input_idx[i].numpy()
        output = boolean_to_string(answer, orig_ids)
        outputs.append(output)
  print (outputs)




In [ ]:
df= pd.DataFrame()
test['test'] = outputs
test.to_csv('model5_rlt.csv')